In [3]:
from os.path import expanduser
import os


def load_env():
    home = expanduser("~")
    print(home)
    with open(home+'/.env', 'r') as f:
        env = dict()
        for line in f.readlines():
            key, value = line.split('=')
            env[key] = value.split('\n')[0]
        return env

def export_env():
    """Loads your .env file and exports the three variables important for mlflow.
    MLFLOW_TRACKING_USERNAME, MLFLOW_TRACKING_PASSWORD, MLFLOW_TRACKING_URI
    """
    env = load_env()
    for key in ["MLFLOW_TRACKING_USERNAME", "MLFLOW_TRACKING_PASSWORD", "MLFLOW_TRACKING_URI"]:
        os.environ[key] = env[key]

/home/erik/miniconda2/envs/automol/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
### RUN FIRST:
### sudo openvpn --config kit.ovpn

import mlflow
from mlflow_utils.load_env import export_env

export_env()
mlflow.set_experiment("demo")

Now for sake of simplicity to demonstrate how mlflow works in conjunction with tensorflow we can work with an example using e.g. the car dataset:

In [6]:
import pandas as pd
import numpy as np

url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

dataset = pd.read_csv(url, names=column_names,
                      na_values='?', comment='\t',
                      sep=' ', skipinitialspace=True)
dataset = dataset.dropna()
# One-hot encode origin:
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Europe,Japan,USA
393,27.0,4,140.0,86.0,2790.0,15.6,82,0,0,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,1,0,0
395,32.0,4,135.0,84.0,2295.0,11.6,82,0,0,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,0,0,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,0,0,1


Let's split it into train and test datasets:

In [7]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')

train_dataset.describe().transpose()[['mean', 'std']]

/home/erik/miniconda2/envs/automol/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,mean,std
MPG,23.310510,7.728652
Cylinders,5.477707,1.699788
Displacement,195.318471,104.331589
Horsepower,104.869427,38.096214
Weight,2990.251592,843.898596
Acceleration,15.559236,2.789230
Model Year,75.898089,3.675642
Europe,0.178344,0.383413
Japan,0.197452,0.398712
USA,0.624204,0.485101


...and fit a normalizer on the train data:

In [8]:
from tensorflow.keras.layers.experimental import preprocessing

normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))

horsepower = np.array(train_features['Horsepower'])
horsepower_normalizer = preprocessing.Normalization(input_shape=[1,])
horsepower_normalizer.adapt(horsepower)

/home/erik/miniconda2/envs/automol/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
from sklearn.neural_network import MLPRegressor
from pprint import pprint

from sklearn.model_selection import learning_curve
from sklearn.neural_network import MLPRegressor

import matplotlib.pyplot as plt
from sklearn.model_selection import ShuffleSplit

model = MLPRegressor(random_state=1, max_iter=300)

def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : estimator instance
        An estimator instance implementing `fit` and `predict` methods which
        will be cloned for each validation.

    title : str
        Title for the chart.

    X : array-like of shape (n_samples, n_features)
        Training vector, where ``n_samples`` is the number of samples and
        ``n_features`` is the number of features.

    y : array-like of shape (n_samples) or (n_samples, n_features)
        Target relative to ``X`` for classification or regression;
        None for unsupervised learning.

    axes : array-like of shape (3,), default=None
        Axes to use for plotting the curves.

    ylim : tuple of shape (2,), default=None
        Defines minimum and maximum y-values plotted, e.g. (ymin, ymax).

    cv : int, cross-validation generator or an iterable, default=None
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, default=None
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like of shape (n_ticks,), dtype={int, float}
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the ``dtype`` is float, it is regarded
        as a fraction of the maximum size of the training set (that is
        determined by the selected validation method), i.e. it has to be within
        (0, 1]. Otherwise it is interpreted as absolute sizes of the training
        sets. Note that for classification the number of samples usually have
        to be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

fig, axes = plt.subplots(3, 1, figsize=(10, 15))

#X, y = load_digits(return_X_y=True)

title = "Learning Curves (MLPRegressor)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

plot_learning_curve(model, title, X, y, axes=axes[:, 0], ylim=(0.7, 1.01),
                    cv=cv, n_jobs=4)

plt.show()

def fetch_logged_data(run_id):
    client = mlflow.tracking.MlflowClient()
    data = client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
    return data.params, data.metrics, tags, artifacts

mlflow.sklearn.autolog()

# train a model
model = MLPRegressor(random_state=1, max_iter=300)
with mlflow.start_run() as run:
    #model.fit(train_features, train_labels)
    train_sizes, train_scores, valid_scores = learning_curve(
        model, train_features, train_labels, cv=5)

# fetch logged data
params, metrics, tags, artifacts = fetch_logged_data(run.info.run_id)

pprint(params)

pprint(metrics)

pprint(tags)

pprint(artifacts)

/home/erik/miniconda2/envs/automol/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/erik/miniconda2/envs/automol/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
2021/01/17 12:47:25 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under sftp://lsdf/kit/iti/projects/aimat-mlflow/artifacts/1/923ccce1a2f74636900af5353f20951e/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above a

/home/erik/miniconda2/envs/automol/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
2021/01/17 12:47:40 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under sftp://lsdf/kit/iti/projects/aimat-mlflow/artifacts/1/923ccce1a2f74636900af5353f20951e/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.
2021/01/17 12:47:41 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under sftp://lsdf/kit/iti

{'activation': 'relu',
 'alpha': '0.0001',
 'batch_size': 'auto',
 'beta_1': '0.9',
 'beta_2': '0.999',
 'early_stopping': 'False',
 'epsilon': '1e-08',
 'hidden_layer_sizes': '(100,)',
 'learning_rate': 'constant',
 'learning_rate_init': '0.001',
 'max_fun': '15000',
 'max_iter': '300',
 'momentum': '0.9',
 'n_iter_no_change': '10',
 'nesterovs_momentum': 'True',
 'power_t': '0.5',
 'random_state': '1',
 'shuffle': 'True',
 'solver': 'adam',
 'tol': '0.0001',
 'validation_fraction': '0.1',
 'verbose': 'False',
 'warm_start': 'False'}
{'training_mae': 4.8401171397364005,
 'training_mse': 41.00441847856583,
 'training_r2_score': 0.3074023565376697,
 'training_rmse': 6.403469253347425,
 'training_score': 0.3074023565376697}
{'estimator_class': 'sklearn.neural_network._multilayer_perceptron.MLPRegressor',
 'estimator_name': 'MLPRegressor'}
['model/MLmodel', 'model/conda.yaml', 'model/model.pkl']


In [16]:
train_sizes

/home/erik/miniconda2/envs/automol/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([ 25,  81, 138, 194, 251])

In [17]:
train_scores

array([[-171.78818781, -137.35200079, -137.35200079, -137.35200079,
        -137.35200079],
       [-155.59015601, -160.67044981, -159.00171333, -159.00171333,
        -159.00171333],
       [-171.46577318, -177.83296987, -166.02167672, -154.20388169,
        -154.20388169],
       [-155.96027267, -160.13302048, -152.70824075, -158.95715633,
        -162.91442758],
       [   0.34820501,    0.64178324,    0.34951227,    0.70090478,
           0.30740236]])

In [18]:
valid_scores

array([[-152.18313773, -160.15583966, -168.46841666, -185.32287514,
        -161.65021322],
       [-151.76320611, -157.20093894, -167.09949215, -183.46834409,
        -160.29820073],
       [-158.93888681, -161.93859903, -165.95771744, -179.85543093,
        -157.19460591],
       [-149.92911489, -152.31898382, -155.8091041 , -182.66591137,
        -160.65226928],
       [   0.39303514,    0.61276656,    0.338383  ,    0.52974886,
           0.39586933]])

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_and_compile_model(norm):
    model = keras.Sequential([
        norm,
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.001))
    return model

In [17]:
class MlFlowCallback(tf.keras.callbacks.Callback):
    """ This Callback logs train and validation metrics to mlflow on every epoch end.
    """
    def on_epoch_end(self, epoch, logs=None):
        mlflow.log_metrics(metrics=logs, step=epoch)
        mlflow.log_metric(key="quality", value=2*epoch, step=epoch)

callbacks = [MlFlowCallback()]

In [18]:
mlflow.tensorflow.autolog()
model = build_and_compile_model(normalizer)

with mlflow.start_run():
    model.fit(train_features, train_labels,
              epochs=10,
              validation_data=(test_features, test_labels),
              callbacks=callbacks)

Epoch 1/10
10/10 [==============================] - 1s 65ms/step - loss: 22.6258 - val_loss: 22.7580
Epoch 2/10
10/10 [==============================] - 1s 74ms/step - loss: 21.6502 - val_loss: 21.6599
Epoch 3/10
10/10 [==============================] - 1s 55ms/step - loss: 20.4286 - val_loss: 20.2192
Epoch 4/10
10/10 [==============================] - 1s 53ms/step - loss: 18.7620 - val_loss: 18.1932
Epoch 5/10
10/10 [==============================] - 1s 53ms/step - loss: 16.4117 - val_loss: 15.4091
Epoch 6/10
10/10 [==============================] - 1s 55ms/step - loss: 13.3561 - val_loss: 12.2330
Epoch 7/10
10/10 [==============================] - 1s 60ms/step - loss: 10.1814 - val_loss: 8.8585
Epoch 8/10
10/10 [==============================] - 1s 57ms/step - loss: 7.1841 - val_loss: 5.6427
Epoch 9/10
10/10 [==============================] - 1s 65ms/step - loss: 5.6648 - val_loss: 4.8275
Epoch 10/10
10/10 [==============================] - 1s 56ms/step - loss: 4.6762 - val_loss: 3.7

Now we can head to [http://141.3.29.7](http://141.3.29.7), log in with our user data and 